<a href="https://colab.research.google.com/github/roshanedachali/TwitterAnalysis/blob/main/HashTagAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [30]:
#IMPORT
import tweepy
import csv
! pip install jsonpickle
import jsonpickle
API_KEY="Q76360NUPSaax6Ic1MyPCqPhD"
API_SECRET="Ef5NJGN1GVJ6hwjOgh7Tv5NtOrBgipvE2umyrP37qkh8XWbOmC"
ACCESS_TOKEN="3180237764-o2OUouxEnigruw79ULQ2RrK78zbONdduBgScexi"
ACCESS_TOKEN_SECRET="yXCX26guHINaRHFCrlJmTTiSweTnlOTd67DDYdbJceQpl"
auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

# Get and Store Tweets


In [31]:
tweetsPerQuery = 100 #this is the maximum provided by API
max_tweets = 100000000 # just for the sake of While loop

# No sinceId and max_id ..Get whathever you have exhaustively
since_id = None
max_id = -1
tweet_count = 0

search_query="#SwachhBharat"
x=0
with open("sb_06_10_21.txt",'w') as f:
    print("Downloading hashtag: " + search_query)
    while(tweet_count<max_tweets):
        try:
            if(max_id<=0):
                if(not since_id):
                    new_tweets = api.search(q=search_query,count=tweetsPerQuery,lang="en",tweet_mode='extended')

                else:
                    new_tweets = api.search(q=search_query,count=tweetsPerQuery,lang="en",tweet_mode='extended',since_id=since_id)
            else:
                if(not since_id):
                    new_tweets = api.search(q=search_query,count=tweetsPerQuery,lang="en",tweet_mode='extended',max_id=str(max_id-1))
                else:
                    new_tweets = api.search(q=search_query,count=tweetsPerQuery,lang="en",tweet_mode='extended',max_id=str(max_id-1),since_id=since_id)

            # Tweets Exhausted
            if(not new_tweets):
                break
            # write all the new_tweets to a json file
            for tweet in new_tweets:
                f.write(jsonpickle.encode(tweet._json,unpicklable=False)+'\n')
                tweet_count+=len(new_tweets)
                max_id=new_tweets[-1].id
        # in case of any error
        except tweepy.TweepError as e:
                print("Some error!!:"+str(e))
                break

print("A total of {0} tweets were downloaded and saved.".format(tweet_count))



A total of 234935 tweets were downloaded and saved.


# Editing the CSV

In [32]:
import json
import csv
import io



f = open('som1.csv','a',encoding='utf-8')
csvWriter = csv.writer(f)
headers=['full_text','retweet_count','user_followers_count','favorite_count','place','coordinates','geo','created_at','id_str']
csvWriter.writerow(headers)

for inputFile in ['sb_10_06_21.txt']:#all the text-file names you want to convert to Csv in the sae folder as this code
    tweets = []
    for line in open("sb_06_10_21.txt", 'r'):
        tweets.append(json.loads(line))

    print('HI',len(tweets))
    count_lines=0
    for tweet in tweets:
        try:
            csvWriter.writerow([tweet['full_text'],tweet['retweet_count'],tweet['user']['followers_count'],tweet['favorite_count'],tweet['place'],tweet['coordinates'],tweet['geo'],tweet['created_at'],str(tweet['id_str'])])
            count_lines+=1
        except Exception as e:
            print(e)
    print(count_lines)
    
import pandas as pd
df = pd.read_csv('som1.csv', encoding = 'unicode_escape')
df.head(5)


HI 2453
2453


,full_text,retweet_count,user_followers_count,favorite_count,place,coordinates,geo,created_at,id_str
0,RT @WeMeanToClean: #WasteSegregation is the ke...,1,1163,0,NaN,NaN,NaN,Fri Jun 11 04:05:14 +0000 2021,1403201634381881351
1,#WasteSegregation is the key to effective #Was...,1,2198,2,NaN,NaN,NaN,Fri Jun 11 04:00:41 +0000 2021,1403200488741421058
2,RT @dpradhanbjp: India has transitioned from w...,228,126,0,NaN,NaN,NaN,Fri Jun 11 03:47:37 +0000 2021,1403197200318959617
3,"RT @SwatiBhalla23: Plant a tree, plant a life!...",2,820,0,NaN,NaN,NaN,Fri Jun 11 03:46:31 +0000 2021,1403196922492600321
4,RT @ManishKhurana: Another one of our voluntee...,2,820,0,NaN,NaN,NaN,Fri Jun 11 03:46:21 +0000 2021,1403196883548450816


# Cleaning

In [33]:
print(len(df.index))
serlis=df.duplicated().tolist()
print(serlis.count(True))
serlis=df.duplicated(['full_text']).tolist()
print(serlis.count(True))
df=df.drop_duplicates(['full_text'])
df.head(5)
index = (df.index)
import re
i = 0
j = 0
while ((i+1) <= len(index)):
    if (i == index[j]):
      txt = df.loc[i]["full_text"]
      txt=re.sub(r'@[A-Z0-9a-z_:]+','',txt)#replace username-tags
      txt=re.sub(r'^[RT]+','',txt)#replace RT-tags
      txt = re.sub('https?://[A-Za-z0-9./]+','',txt)#replace URLs
      txt=re.sub("[^a-zA-Z]", " ",txt)#replace hashtags
      df.at[i,"full_text"]=txt
      i += 1
      j += 1
    else: 
      i += 1
df = df.drop("id_str", axis = 1)
df.head()


4886
1833
3998


,full_text,retweet_count,user_followers_count,favorite_count,place,coordinates,geo,created_at
0,WasteSegregation is the key to effective W...,1,1163,0,NaN,NaN,NaN,Fri Jun 11 04:05:14 +0000 2021
1,WasteSegregation is the key to effective Was...,1,2198,2,NaN,NaN,NaN,Fri Jun 11 04:00:41 +0000 2021
2,India has transitioned from women s develo...,228,126,0,NaN,NaN,NaN,Fri Jun 11 03:47:37 +0000 2021
3,Plant a tree plant a life WeMeanToClean...,2,820,0,NaN,NaN,NaN,Fri Jun 11 03:46:31 +0000 2021
4,Another one of our volunteer sets up home nu...,2,820,0,NaN,NaN,NaN,Fri Jun 11 03:46:21 +0000 2021


# Sentiment Analysis

In [51]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer, PorterStemmer
from wordcloud import WordCloud, STOPWORDS
from textblob import TextBlob
nltk.download('stopwords')
senti_score = []
df.head()

## Change the reviews type to string
df['full_text'] = df['full_text'].astype(str)

## Before lowercasing 
df['full_text'][2]

## Lowercase all reviews
df['full_text'] = df['full_text'].apply(lambda x: " ".join(x.lower() for x in x.split()))

## Remove special characters
bad_chars = [';', ':', '!', "*", "@", "$", "^","&","(",")","?","=","/","%"] 
for i in bad_chars :
    df = df.replace(i, '')

## Remove stop words 
stop = stopwords.words('english')
df['full_text'] = df['full_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

## Stemming
st = PorterStemmer()
df['full_text'] = df['full_text'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

## Sentiment Analysis
def senti(x):
    return TextBlob(x).sentiment  

df['senti_score'] = df['full_text'].apply(senti)

#df.drop('geo', inplace=True, axis=1)

df.head()
#(Sentiment Score, Subjectivity Score)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,full_text,created_at,senti_score
0,wastesegreg key effect wastemanag wemeantoclea...,Fri Jun 11 04:05:14 +0000 2021,"(0.0, 1.0)"
1,wastesegreg key effect wastemanag wemeantoclea...,Fri Jun 11 04:00:41 +0000 2021,"(0.0, 1.0)"
2,india transit women develop women led develop ...,Fri Jun 11 03:47:37 +0000 2021,"(0.0, 0.0)"
3,plant tree plant life wemeantoclean cleandelhi...,Fri Jun 11 03:46:31 +0000 2021,"(0.0, 0.0)"
4,anoth one volunt set home nurseri nativetr gre...,Fri Jun 11 03:46:21 +0000 2021,"(0.0, 0.0)"
